In [1]:
from typing import SupportsFloat

import numpy as np
import numpy.typing as npt
import pandas as pd
from numpy.linalg import LinAlgError

# Hiperparâmetros

In [2]:
# Valores de n utilizados
TESTED_N = [2, 4, 6, 8, 10, 20, 40, 60, 80, 100, 150, 200]

# Geração de matrizes

In [3]:
def generate_pascal(n: int) -> npt.NDArray:
    mat = np.zeros((n, n))
    for i in range(n):
        mat[0, i] = 1
        mat[i, 0] = 1
    for i in range(1, n):
        for j in range(1, n):
            mat[i, j] = mat[i - 1, j] + mat[i, j - 1]
    return mat

In [4]:
pascal_matrices = {n: generate_pascal(n) for n in TESTED_N}

In [5]:
def ill_conditioned_matrix(n: int):
    random_entries = np.random.rand(n, n)
    fourth_power = np.linalg.matrix_power(random_entries, 4)
    return fourth_power

# Funções de erro

In [6]:
def calculate_k2(matrix: npt.NDArray):
    ps_inv = np.linalg.inv(matrix.T @ matrix) @ matrix.T
    return np.linalg.matrix_norm(matrix, ord=2) * np.linalg.matrix_norm(ps_inv, ord=2)

In [7]:
def residual(original_matrix: npt.NDArray, algo_result: npt.NDArray) -> SupportsFloat:
    n = min(original_matrix.shape)
    id_mat = np.identity(n)
    num = np.linalg.matrix_norm((algo_result @ original_matrix) - id_mat, ord=2)
    den = np.linalg.matrix_norm(original_matrix, ord=2) * np.linalg.matrix_norm(algo_result, ord=2)
    return num/den

In [8]:
# Machine epsilon
eps = np.finfo(float).eps
def stability_factor(original_matrix: npt.NDArray, algo_result) -> SupportsFloat:
    ps_inv = np.linalg.inv(original_matrix.T @ original_matrix) @ original_matrix.T
    k2 = calculate_k2(original_matrix)
    num = np.linalg.matrix_norm(algo_result - ps_inv, ord=2)
    den = eps * np.linalg.matrix_norm(ps_inv, ord=2) * k2
    return num / den

# Construção dos resultados

In [124]:
def create_error_df(
    decomposition: str,
    columns: dict[str, list]
):
    df = pd.DataFrame({
        "n": columns.get("n", []),
        "k2": columns.get("k2", []),
        "decomposition": decomposition,
        "matrix_type": columns.get("matrix_type", []),
        "stability_factor": columns.get("stability_factor", []),
        "residual": columns.get("residual", [])
    })
    return df

In [133]:
dfs = []

# QR

In [125]:
n_list = []
k2_list = []
matrix_type_list = []
stability_factor_list = []
residual_list = []

Pascal

In [126]:
for n in TESTED_N:
    A = pascal_matrices[n]
    stop = False
    try:
        Q, R = np.linalg.qr(A)
        psiinv_qr = np.linalg.inv(R) @ Q.T
        res = residual(A, psiinv_qr)
        stab = stability_factor(A, psiinv_qr)
    except LinAlgError:
        res = np.nan
        stop = True
    n_list.append(n)
    k2_list.append(calculate_k2(A))
    matrix_type_list.append("Pascal")
    stability_factor_list.append(stab)
    residual_list.append(res)
    if stop:
        break

Aleatória

In [127]:
for n in TESTED_N:
    A = np.random.rand(3 * n, n)
    stop = False
    try:
        Q, R = np.linalg.qr(A)
        psiinv_qr = np.linalg.inv(R) @ Q.T
        res = residual(A, psiinv_qr)
        stab = stability_factor(A, psiinv_qr)
    except LinAlgError as exception:
        print(f"Erro para n={n}: {exception}")
        res = np.nan
        stop = True
    n_list.append(n)
    k2_list.append(calculate_k2(A))
    matrix_type_list.append("Aleatória")
    stability_factor_list.append(stab)
    residual_list.append(res)
    if stop:
        break

Aleatória mal-condicionada

In [128]:
for n in TESTED_N:
    A = ill_conditioned_matrix(n)
    stop = False
    try:
        Q, R = np.linalg.qr(A)
        psiinv_qr = np.linalg.inv(R) @ Q.T
        res = residual(A, psiinv_qr)
        stab = stability_factor(A, psiinv_qr)
    except LinAlgError:
        res = np.nan
        stop = True
    n_list.append(n)
    k2_list.append(calculate_k2(A))
    matrix_type_list.append("Aleatória Mal-Condicionada")
    stability_factor_list.append(stab)
    residual_list.append(res)
    if stop:
        break

In [129]:
qr_error_df = create_error_df(
    "QR", {
        "n": n_list,
        "k2": k2_list,
        "matrix_type": matrix_type_list,
        "stability_factor": stability_factor_list,
        "residual": residual_list
    }
)

In [130]:
qr_error_df

,n,k2,decomposition,matrix_type,stability_factor,residual
0,2,6.854102e+00,QR,Pascal,6.802519e-01,5.840249e-17
1,4,6.919374e+02,QR,Pascal,1.677643e+01,4.864344e-17
2,6,1.107867e+05,QR,Pascal,6.301025e+02,1.060905e-17
3,8,2.064579e+07,QR,Pascal,6.559253e+03,5.679337e-17
4,10,1.511318e+09,QR,Pascal,1.115364e+07,4.931437e-17
5,20,3.392960e+13,QR,Pascal,5.520517e+08,2.126271e-18
6,40,3.698571e+26,QR,Pascal,5.803057e-08,6.424184e-16
7,60,3.297535e+38,QR,Pascal,2.054184e-21,4.493240e-13
8,80,3.115602e+50,QR,Pascal,8.426599e-35,3.826057e-10
9,100,5.496571e+62,QR,Pascal,1.409705e-47,3.092594e-10


No caso da matriz de Pascal, o óbvio aumento do $k_2$ causou uma redução muito alta no fator de estabilidade.

No caso das matrizes aleatórias, mesmo a mal-condicionada acaba tendo um resíduo muito baixo, e alto fator de estabilidade.

In [134]:
dfs.append(qr_error_df)

# Cholesky

In [135]:
n_list = []
k2_list = []
matrix_type_list = []
stability_factor_list = []
residual_list = []

In [136]:
for n in TESTED_N:
    A = pascal_matrices[n]
    stop = False
    try:
        M = A.T @ A
        R = np.linalg.cholesky(M, upper=True)
        Y = np.linalg.solve(R.T, A.T)
        psiinv_chol = np.linalg.solve(R, Y)
        res = residual(A, psiinv_chol)
        stab = stability_factor(A, psiinv_chol)
    except LinAlgError:
        res = np.nan
        stop = True
    n_list.append(n)
    k2_list.append(calculate_k2(A))
    matrix_type_list.append("Pascal")
    stability_factor_list.append(stab)
    residual_list.append(res)
    if stop:
        break

In [137]:
for n in TESTED_N:
    A = np.random.rand(3 * n, n)
    stop = False
    try:
        M = A.T @ A
        R = np.linalg.cholesky(M, upper=True)
        Y = np.linalg.solve(R.T, A.T)
        psiinv_chol = np.linalg.solve(R, Y)
        res = residual(A, psiinv_chol)
        stab = stability_factor(A, psiinv_chol)
    except LinAlgError:
        res = np.nan
        stop = True
    n_list.append(n)
    k2_list.append(calculate_k2(A))
    matrix_type_list.append("Aleatória")
    stability_factor_list.append(stab)
    residual_list.append(res)
    if stop:
        break

In [138]:
for n in TESTED_N:
    A = ill_conditioned_matrix(n)
    stop = False
    try:
        M = A.T @ A
        R = np.linalg.cholesky(M, upper=True)
        Y = np.linalg.solve(R.T, A.T)
        psiinv_chol = np.linalg.solve(R, Y)
        res = residual(A, psiinv_chol)
        stab = stability_factor(A, psiinv_chol)
    except LinAlgError:
        res = np.nan
        stop = True
    n_list.append(n)
    k2_list.append(calculate_k2(A))
    matrix_type_list.append("Aleatória Mal-Condicionada")
    stability_factor_list.append(stab)
    residual_list.append(res)
    if stop:
        break

In [139]:
chol_error_df = create_error_df(
    "Cholesky", {
        "n": n_list,
        "k2": k2_list,
        "matrix_type": matrix_type_list,
        "stability_factor": stability_factor_list,
        "residual": residual_list
    }
)

In [140]:
chol_error_df

,n,k2,decomposition,matrix_type,stability_factor,residual
0,2,6.854102e+00,Cholesky,Pascal,1.827855e+00,2.829092e-16
1,4,6.919374e+02,Cholesky,Pascal,1.098630e+01,1.345211e-14
2,6,1.107867e+05,Cholesky,Pascal,2.218880e+02,1.290553e-12
3,8,2.064579e+07,Cholesky,Pascal,4.703877e+04,1.502158e-10
4,10,1.511318e+09,Cholesky,Pascal,1.222902e+07,2.880003e-08
5,20,3.392960e+13,Cholesky,Pascal,1.222902e+07,NaN
6,2,3.883699e+00,Cholesky,Aleatória,3.756308e-01,2.066007e-16
7,4,4.621184e+00,Cholesky,Aleatória,3.304749e-01,1.369035e-16
8,6,1.218956e+01,Cholesky,Aleatória,7.824608e-01,3.571127e-16
9,8,8.371458e+00,Cholesky,Aleatória,9.083222e-01,4.878683e-16


In [ ]:
dfs.append(chol_error_df)

# SVD

In [142]:
n_list = []
k2_list = []
matrix_type_list = []
stability_factor_list = []
residual_list = []

In [143]:
for n in TESTED_N:
    A = pascal_matrices[n]
    stop = False
    try:
        U, S, Vt = np.linalg.svd(A, full_matrices=False)
        V = Vt.T
        psiinv_svd = V @ np.diag(S ** -1) @ U.T
        res = residual(A, psiinv_svd)
    except LinAlgError:
        res = np.nan
        stop = True
    n_list.append(n)
    k2_list.append(calculate_k2(A))
    matrix_type_list.append("Pascal")
    stability_factor_list.append(stab)
    residual_list.append(res)
    if stop:
        break

In [144]:
for n in TESTED_N:
    A = np.random.rand(3 * n, n)
    stop = False
    try:
        U, S, Vt = np.linalg.svd(A, full_matrices=False)
        V = Vt.T
        psiinv_svd = V @ np.diag(S ** -1) @ U.T
        res = residual(A, psiinv_svd)
    except LinAlgError:
        res = np.nan
        stop = True
    n_list.append(n)
    k2_list.append(calculate_k2(A))
    matrix_type_list.append("Aleatória")
    stability_factor_list.append(stab)
    residual_list.append(res)
    if stop:
        break

In [145]:
for n in TESTED_N:
    A = ill_conditioned_matrix(n)
    stop = False
    try:
        U, S, Vt = np.linalg.svd(A, full_matrices=False)
        V = Vt.T
        psiinv_svd = V @ np.diag(S ** -1) @ U.T
        res = residual(A, psiinv_svd)
    except LinAlgError:
        res = np.nan
        stop = True
    n_list.append(n)
    k2_list.append(calculate_k2(A))
    matrix_type_list.append("Aleatória Mal-Condicionada")
    stability_factor_list.append(stab)
    residual_list.append(res)
    if stop:
        break

In [146]:
svd_error_df = create_error_df(
    "SVD", {
        "n": n_list,
        "k2": k2_list,
        "matrix_type": matrix_type_list,
        "stability_factor": stability_factor_list,
        "residual": residual_list
    }
)

In [147]:
svd_error_df

,n,k2,decomposition,matrix_type,stability_factor,residual
0,2,6.854102e+00,SVD,Pascal,1.014676e+06,1.289983e-16
1,4,6.919374e+02,SVD,Pascal,1.014676e+06,7.187024e-17
2,6,1.107867e+05,SVD,Pascal,1.014676e+06,2.661360e-17
3,8,2.064579e+07,SVD,Pascal,1.014676e+06,4.233653e-17
4,10,1.511318e+09,SVD,Pascal,1.014676e+06,1.396618e-17
5,20,3.392960e+13,SVD,Pascal,1.014676e+06,2.366071e-17
6,40,3.698571e+26,SVD,Pascal,1.014676e+06,2.865766e-16
7,60,3.297535e+38,SVD,Pascal,1.014676e+06,4.107881e-16
8,80,3.115602e+50,SVD,Pascal,1.014676e+06,4.846123e-16
9,100,5.496571e+62,SVD,Pascal,1.014676e+06,3.479979e-16


Não apresentou nenhum alto resíduo para nenhum tipo de matriz. Método mais estável dentre os apresentados.

In [ ]:
dfs.append(svd_error_df)

# Outras decomposições

## ST

Essa versão não garante `S` como positivo definido

In [149]:
def __verify_st_conditions(A: npt.NDArray):
    if np.linalg.det(A) == 0:
        raise ValueError("Matrix is singular")
    
    # if np.allclose(A, A.T):
    #     raise ValueError("Matrix is symmetric")

    n = A.shape[0]

    if (n != A.shape[1]):
        raise ValueError("Matrix is not square")
    
    for k in range(1, n + 1):
        if np.linalg.det(A[:k, :k]) == 0:
            raise ValueError("Matrix does not satisfy the principal minor condition")


def st_simple_decompose(A: npt.NDArray):
    """
    Decomposes A into S and T matrices, where S is symmetrical and T is triangular.

    In this case, S is **NOT** guaranteed to be positive definite.
    """
    __verify_st_conditions(A)

    S = np.zeros_like(A, dtype=np.float64)
    T = np.zeros_like(A, dtype=np.float64)
    n = A.shape[0]

    # Recursion base case
    # T
    T[0, 0] = 1
    T[0, 1] = (A[0, 1] - A[1, 0]) / A[0, 0]
    T[1, 1] = 1
    T[1, 0] = 0

    # S
    S[0, 0] = A[0, 0]
    S[0, 1] = A[1, 0]
    S[1, 0] = A[1, 0]
    S[1, 1] = A[1, 1] - A[1, 0] * T[0, 1]

    for k in range(2, n):
        Tk = T[:k, :k]
        T[k, k] = 1
        s = np.linalg.inv(Tk.T) @ A[k, :k]
        S[k, :k] = s
        S[:k, k] = s
        t = np.linalg.inv(S[:k, :k]) @ (A[:k, k] - S[:k, k])
        T[:k, k] = t
        S[k, k] = A[k, k] - (s @ t)
    return S, T

In [150]:
n_list = []
k2_list = []
matrix_type_list = []
stability_factor_list = []
residual_list = []

In [151]:
for n in TESTED_N:
    A = pascal_matrices[n]
    stop = False
    try:
        S, T = st_simple_decompose(A)
        psiinv_st = np.linalg.inv(T) @ np.linalg.inv(S)
        res = residual(A, psiinv_st)
        stab = stability_factor(A, psiinv_st)
    except LinAlgError:
        res = np.nan
        stop = True
    n_list.append(n)
    k2_list.append(calculate_k2(A))
    matrix_type_list.append("Pascal")
    stability_factor_list.append(stab)
    residual_list.append(res)
    if stop:
        break

e:\Repositories\alcd\.venv\Lib\site-packages\numpy\linalg\_linalg.py:2430: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [152]:
for n in TESTED_N:
    A = np.random.rand(3 * n, n)
    stop = False
    try:
        M = A.T @ A
        S, T = st_simple_decompose(M)
        psiinv_st = np.linalg.inv(T) @ np.linalg.inv(S) @ A.T
        res = residual(A, psiinv_st)
        stab = stability_factor(A, psiinv_st)
    except LinAlgError:
        res = np.nan
        stop = True
    n_list.append(n)
    k2_list.append(calculate_k2(A))
    matrix_type_list.append("Aleatória")
    stability_factor_list.append(stab)
    residual_list.append(res)
    if stop:
        break

In [153]:
for n in TESTED_N:
    A = ill_conditioned_matrix(n)
    stop = False
    try:
        S, T = st_simple_decompose(A)
        psiinv_st = np.linalg.inv(T) @ np.linalg.inv(S)
        res = residual(A, psiinv_st)
        stab = stability_factor(A, psiinv_st)
    except LinAlgError:
        res = np.nan
        stop = True
    n_list.append(n)
    k2_list.append(calculate_k2(A))
    matrix_type_list.append("Aleatória Mal-Condicionada")
    stability_factor_list.append(stab)
    residual_list.append(res)
    if stop:
        break

In [154]:
st_error_df = create_error_df(
    "ST", {
        "n": n_list,
        "k2": k2_list,
        "matrix_type": matrix_type_list,
        "stability_factor": stability_factor_list,
        "residual": residual_list
    }
)

In [156]:
st_error_df

,n,k2,decomposition,matrix_type,stability_factor,residual
0,2,6.854102e+00,ST,Pascal,5.835921e-01,0.000000e+00
1,4,6.919374e+02,ST,Pascal,1.685164e+01,7.075863e-17
2,6,1.107867e+05,ST,Pascal,6.300913e+02,2.152154e-16
3,8,2.064579e+07,ST,Pascal,6.559264e+03,2.158877e-16
4,10,1.511318e+09,ST,Pascal,1.115364e+07,7.454864e-16
5,20,3.392960e+13,ST,Pascal,4.263146e+08,6.378197e-13
6,40,3.698571e+26,ST,Pascal,2.717815e-09,1.267599e-09
7,60,3.297535e+38,ST,Pascal,1.343549e-21,1.355412e-08
8,80,3.115602e+50,ST,Pascal,2.624811e-35,6.852727e-08
9,100,5.496571e+62,ST,Pascal,8.199782e-48,8.079343e-07


In [157]:
dfs.append(st_error_df)

## TS

In [100]:
def ts(A):
    __verify_st_conditions(A)

    n = A.shape[0]

    T = np.zeros((n, n))
    S = np.zeros((n, n))
    L = np.zeros((n, n))
    Lt = np.zeros((n, n))

    detA = np.linalg.det(A[:2,:2])
    T[0, 0] = A[0, 0]
    T[0, 1] = 0
    T[1, 0] = A[1, 0] - (A[0, 1] * detA)/A[0, 0]
    T[1, 1] = detA

    S[0, 0] = 1
    S[0, 1] = A[0, 1]/A[0, 0]
    S[1, 0] = A[0, 1]/A[0, 0]
    S[1, 1] = (1/A[0, 0]) + (A[0, 1]/A[0, 0])**2

    L[:2,:2] = np.linalg.cholesky(S[:2,:2])

    Lt[:2,:2] = L[:2,:2].T
    
    for k in range(2, n):
        t_inv = np.linalg.inv(T[:k,:k])
        l_inv = np.linalg.inv(L[:k,:k])
        lt_inv = np.linalg.inv(Lt[:k,:k])

        l = l_inv @ t_inv @ A[:k, k]
        L[:k,k] = 0
        L[k, :k] = l
        Lt[:k,k] = l
        Lt[k, :k] = 0

        beta = 1
        if ((A[k, k] - (A[k, :k] @ lt_inv @ l_inv @ t_inv @ A[:k, k])) / beta) < 0:
            beta = -beta
        tau = np.sqrt((A[k, k] - (A[k, :k] @ lt_inv @ l_inv @ t_inv @ A[:k, k])) / beta)

        t = (A[k, :k] @ lt_inv @ l_inv) - (beta * l @ l_inv)
        T[:k, k] = 0
        T[k, :k] = t

        T[k, k] = beta
        L[k, k] = tau
        Lt[k, k] = tau
    S = L @ Lt
    return T, S

In [158]:
n_list = []
k2_list = []
matrix_type_list = []
stability_factor_list = []
residual_list = []

In [159]:
for n in TESTED_N:
    A = pascal_matrices[n]
    stop = False
    try:
        T, S = ts(A)
        psiinv_st = np.linalg.inv(S) @ np.linalg.inv(T)
        res = residual(A, psiinv_st)
        stab = stability_factor(A, psiinv_st)
    except LinAlgError:
        res = np.nan
        stop = True
    n_list.append(n)
    k2_list.append(calculate_k2(A))
    matrix_type_list.append("Pascal")
    stability_factor_list.append(stab)
    residual_list.append(res)
    if stop:
        break

e:\Repositories\alcd\.venv\Lib\site-packages\numpy\linalg\_linalg.py:2430: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [160]:
for n in TESTED_N:
    A = np.random.rand(3 * n, n)
    stop = False
    try:
        M = A.T @ A
        S, T = ts(M)
        psiinv_st = np.linalg.inv(S) @ np.linalg.inv(T) @ A.T
        res = residual(A, psiinv_st)
        stab = stability_factor(A, psiinv_st)
    except LinAlgError:
        res = np.nan
        stop = True
    n_list.append(n)
    k2_list.append(calculate_k2(A))
    matrix_type_list.append("Aleatória")
    stability_factor_list.append(stab)
    residual_list.append(res)
    if stop:
        break

In [161]:
for n in TESTED_N:
    A = ill_conditioned_matrix(n)
    stop = False
    try:
        S, T = ts(A)
        psiinv_st = np.linalg.inv(S) @ np.linalg.inv(T)
        res = residual(A, psiinv_st)
        stab = stability_factor(A, psiinv_st)
    except LinAlgError:
        res = np.nan
        stop = True
    n_list.append(n)
    k2_list.append(calculate_k2(A))
    matrix_type_list.append("Aleatória Mal-Condicionada")
    stability_factor_list.append(stab)
    residual_list.append(res)
    if stop:
        break

In [162]:
ts_error_df = create_error_df(
    "TS", {
        "n": n_list,
        "k2": k2_list,
        "matrix_type": matrix_type_list,
        "stability_factor": stability_factor_list,
        "residual": residual_list
    }
)

In [163]:
ts_error_df

,n,k2,decomposition,matrix_type,stability_factor,residual
0,2,6.854102e+00,TS,Pascal,5.835921e-01,0.000000e+00
1,4,6.919374e+02,TS,Pascal,1.685164e+01,7.075863e-17
2,6,1.107867e+05,TS,Pascal,6.300913e+02,2.152154e-16
3,8,2.064579e+07,TS,Pascal,6.559241e+03,2.908162e-16
4,10,1.511318e+09,TS,Pascal,1.115364e+07,2.682441e-15
5,20,3.392960e+13,TS,Pascal,4.570436e+08,3.222529e-13
6,40,3.698571e+26,TS,Pascal,2.697590e-06,4.656644e-19
7,60,3.297535e+38,TS,Pascal,2.697590e-06,NaN
8,2,4.418565e+00,TS,Aleatória,5.357089e+14,2.319210e-01
9,4,5.242617e+00,TS,Aleatória,3.292005e+15,5.382135e-01


In [164]:
dfs.append(ts_error_df)

# Resultados totais

In [165]:
full_df = pd.concat(dfs, ignore_index=True)

In [187]:
full_df

,n,k2,decomposition,matrix_type,stability_factor,residual
0,2,6.854102e+00,QR,Pascal,6.802519e-01,5.840249e-17
1,4,6.919374e+02,QR,Pascal,1.677643e+01,4.864344e-17
2,6,1.107867e+05,QR,Pascal,6.301025e+02,1.060905e-17
3,8,2.064579e+07,QR,Pascal,6.559253e+03,5.679337e-17
4,10,1.511318e+09,QR,Pascal,1.115364e+07,4.931437e-17
...,...,...,...,...,...,...
171,60,1.341887e+09,TS,Aleatória Mal-Condicionada,1.604121e+11,2.023161e-03
172,80,1.410797e+09,TS,Aleatória Mal-Condicionada,7.732801e+12,6.091722e-03
173,100,8.525675e+09,TS,Aleatória Mal-Condicionada,1.288510e+11,1.493860e-03
174,150,7.411671e+09,TS,Aleatória Mal-Condicionada,1.393217e+13,1.803642e-03
